In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_from_disk, Dataset
save_path = '/content/drive/My Drive/asr_sinhala/ProcessedData/trained_dataset_6'
processed_dataset = load_from_disk(save_path)


In [ ]:
import torch

def random_frequency_time_masking(features, num_freq_masks=15, num_time_masks=400):
    """
    Randomly masks `num_freq_masks` frequency bins and `num_time_masks` time frames in a non-contiguous manner.
    """
    B, C, F, T = features.shape  # Batch, Channels, Frequency bins, Time frames

    for b in range(B):  # Iterate over each sample in batch
        # Mask random frequency bins
        freq_mask_indices = torch.randperm(F)[:num_freq_masks]
        features[b, :, freq_mask_indices, :] = 0

        # Mask random time frames
        time_mask_indices = torch.randperm(T)[:num_time_masks]
        features[b, :, :, time_mask_indices] = 0

    return features



In [ ]:
def augment_spectrogram(example):
    """
    Applies random frequency and time masking to the spectrogram features.
    Assumes that 'input_features' contains the spectrogram of shape (freq, time).
    """
    if "input_features" in example:
        features = torch.tensor(example["input_features"])
        if len(features.shape) == 3:
            features = features.squeeze(0)

        features = features.unsqueeze(0).unsqueeze(0)
        augmented_features = random_frequency_time_masking(features)

        example["input_features"] = augmented_features.squeeze(0).squeeze(0).numpy().tolist()

    return example

In [ ]:

augmented_dataset = processed_dataset.map(augment_spectrogram)
augmented_dataset.save_to_disk('/content/drive/My Drive/specAugmented_trained_dataset_6')